In [41]:
import os 
from IPython.display import display
import pandas as pd
import sys
sys.path.insert(0, './utils')

from sql_connection import Connector


In [42]:
file_path = './data/water_use_by_country_cleaned.csv'
db_name = 'water_usage.db'

# Создание экземпляра класса Connector и подключение к базе данных
connector = Connector(db_name)
connector.connect()

# Проверка существования файла и импорт данных
if os.path.exists(file_path):
    # Загрузка датасета из CSV файла
    df = pd.read_csv(file_path)

    # Создание таблицы и вставка данных
    connector.cursor.execute('''
    CREATE TABLE IF NOT EXISTS water_usage (
        country TEXT,
        yearly_water_used REAL,
        daily_water_used_per_capita REAL,
        population INTEGER
    )
    ''')

    df.to_sql('water_usage', connector.conn, if_exists='replace', index=False)
    print("Данные успешно импортированы в SQLite базу данных.")
else:
    print(f"Файл {file_path} не найден. Убедитесь, что путь указан правильно.")

    
# Закрытие соединения
connector.close()


Данные успешно импортированы в SQLite базу данных.


In [43]:
query = '''SELECT * FROM water_usage'''
data = connector.execute_query(query)
columns = ['Country', 'yearly_water_used(m3)', 'daily_water_used_per_capita(l)', 'Population']
df = pd.DataFrame(data, columns=columns)
# Просматриваем первые несколько строк датасета
print(df.head())

# Получаем общую информацию о датасете
print(df.info())

# Выводим статистические данные о числовых столбцах
print(df.describe())


               Country  yearly_water_used(m3)  daily_water_used_per_capita(l)  Population
0          Afghanistan            20280000000                            2760    20130327
1              Albania             1311000000                            1177     3050809
2              Algeria             9978000000                             669    40850721
3               Angola              705800000                             100    19291161
4  Antigua and Barbuda               11500000                             361       87260
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country                         179 non-null    object
 1   yearly_water_used(m3)           179 non-null    int64 
 2   daily_water_used_per_capita(l)  179 non-null    int64 
 3   Population                      179 non-null    int

In [44]:
query = '''
    SELECT
        country,
        "Yearly Water Used(m3, thousand of liters)",
        "Daily Water Used Per Capita(liters)",
        population
    FROM water_usage
    LIMIT 10'''
data = connector.execute_query(query)
for row in data:
    print(row)


('Afghanistan', 20280000000, 2760, 20130327)
('Albania', 1311000000, 1177, 3050809)
('Algeria', 9978000000, 669, 40850721)
('Angola', 705800000, 100, 19291161)
('Antigua and Barbuda', 11500000, 361, 87260)
('Argentina', 37780000000, 2480, 41730660)
('Armenia', 2847000000, 2676, 2914963)
('Australia', 16130000000, 1816, 24329963)
('Austria', 3492000000, 1144, 8365092)
('Azerbaijan', 12780000000, 3512, 9968510)


In [45]:
# Настройка ширины вывода в Jupyter Notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)

query = '''
    SELECT
        country,
        "Yearly Water Used(m3, thousand of liters)",
        "Daily Water Used Per Capita(liters)",
        population
    FROM water_usage
    LIMIT 10'''
data = connector.execute_query(query)

# Создание DataFrame из полученных данных
columns = ['Country', 'yearly_water_used', 'daily_water_used_per_capita', 'Population']
df = pd.DataFrame(data, columns=columns)

print(df)

               Country  yearly_water_used  daily_water_used_per_capita  Population
0          Afghanistan        20280000000                         2760    20130327
1              Albania         1311000000                         1177     3050809
2              Algeria         9978000000                          669    40850721
3               Angola          705800000                          100    19291161
4  Antigua and Barbuda           11500000                          361       87260
5            Argentina        37780000000                         2480    41730660
6              Armenia         2847000000                         2676     2914963
7            Australia        16130000000                         1816    24329963
8              Austria         3492000000                         1144     8365092
9           Azerbaijan        12780000000                         3512     9968510


In [46]:
# Вывод топ 5 стран с наименьшим расходом воды на душу населения
query = '''
    SELECT
        country,
        "Yearly Water Used(m3, thousand of liters)",
        "Daily Water Used Per Capita(liters)",
        population
        FROM water_usage
        ORDER BY "Daily Water Used Per Capita(liters)"
        LIMIT 5'''
data = connector.execute_query(query)

columns = ['Country', 'yearly_water_used', 'daily_water_used_per_capita', 'Population']
df = pd.DataFrame(data, columns=columns)

print(df)

                    Country  yearly_water_used  daily_water_used_per_capita  Population
0                  DR Congo          683600000                           32    58775724
1                     Congo           46000000                           38     3350771
2  Central African Republic           72500000                           46     4288810
3                     Benin          130000000                           48     7445596
4                  Maldives            5900000                           48      336170


In [47]:
# Настройка ширины вывода в Jupyter Notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)

# Вывод топ 5 стран с наибольшим расходом воды на душу населения
query = '''
    SELECT
        country,
        "Yearly Water Used(m3, thousand of liters)",
        "Daily Water Used Per Capita(liters)",
        population
        FROM water_usage
        ORDER BY "Daily Water Used Per Capita(liters)" DESC
        LIMIT 5'''
data = connector.execute_query(query)

columns = ['Country', 'yearly_water_used', 'daily_water_used_per_capita', 'Population']
df = pd.DataFrame(data, columns=columns)

print(df)

        Country  yearly_water_used  daily_water_used_per_capita  Population
0  Turkmenistan        27950000000                        15445     4958093
1         Chile        35430000000                         5880    16509195
2        Guyana         1445000000                         5284      749256
3    Uzbekistan        54560000000                         4778    31284568
4    Tajikistan        11490000000                         4460     7057566


In [48]:
query_max = '''
    SELECT
        country,
        MAX("Daily Water Used Per Capita(liters)")
    FROM water_usage
'''

query_min = '''
    SELECT
        country,
        MIN("Daily Water Used Per Capita(liters)")
    FROM water_usage
'''

query_avg = '''
    SELECT
        FLOOR(AVG("Daily Water Used Per Capita(liters)"))
    FROM water_usage
'''


data_max = connector.execute_query(query_max)
data_min = connector.execute_query(query_min)
data_avg = connector.execute_query(query_avg)

columns_max = ['country', 'daily_water_used_by_person']
df_max = pd.DataFrame(data_max, columns=columns_max)

columns_min = ['country', 'daily_water_used_by_person']
df_min = pd.DataFrame(data_min, columns=columns_min)

columns_avg = ['daily_water_used_by_person']
df_avg = pd.DataFrame(data_avg, columns=columns_avg)

print(f'\nmax: {df_max} \nmin: {df_min} \navg: {df_avg}')



max:         country  daily_water_used_by_person
0  Turkmenistan                       15445 
min:     country  daily_water_used_by_person
0  DR Congo                          32 
avg:    daily_water_used_by_person
0                      1329.0


In [49]:
query = '''
    SELECT
        country,
        "Yearly Water Used(m3, thousand of liters)",
        "Daily Water Used Per Capita(liters)",
        population
    FROM water_usage
    WHERE country = 'Belarus'
'''

data = connector.execute_query(query)

columns = ['country', 'yearly_water_used', 'daily_water_used_per_capita', 'population']
df = pd.DataFrame(data, columns=columns)

print(df)

   country  yearly_water_used  daily_water_used_per_capita  population
0  Belarus         1452000000                          419     9499064


In [50]:
query = '''
    SELECT
        SUM("Yearly Water Used(m3, thousand of liters)"),
        SUM("Daily Water Used Per Capita(liters)")
    FROM water_usage
'''

data = connector.execute_query(query)

columns = ['total_yearly_water_used(m3)', 'total_daily_water_used_per_capita(l)']
df = pd.DataFrame(data, columns=columns)

print(df)

   total_yearly_water_used(m3)  total_daily_water_used_per_capita(l)
0                3996757700000                                237976
